### Exemplo: No nosso caso, vamos ver a performance de uma carteira de ativos. Vamos chamar de carteira do Lira.

- Temos o arquivo 'Carteira.xlsx' com os ativos e suas respectivas quantidades
- Vamos analisar como que os ativos performaram, quanto que rendeu a carteira como um todo e comparar com o IBOV

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf

carteira = pd.read_excel('Carteira.xlsx')
carteira = carteira.drop(carteira[carteira['Ativos'] == 'GNDI3'].index)
display(carteira)

### Criando nosso dataframe de Cotações dos ativos da carteira

In [ ]:
cotacoes_carteira = pd.DataFrame()

start = dt.datetime(2020, 1, 1)
end = dt.datetime(2020, 11, 10)

for ativo in carteira['Ativos']:
    if ativo == 'GNDI3':
        pass
    else:
        cotacoes_carteira[ativo] = yf.download(f'{ativo}.SA', start, end)['Adj Close']
    
display(cotacoes_carteira)

### Será que todos os dados vieram corretos?

In [ ]:
cotacoes_carteira.info()

### Ajustando os dados

In [ ]:
# df_media = cotacoes_carteira.mean()
# cotacoes_carteira = cotacoes_carteira.fillna(df_media)
cotacoes_carteira = cotacoes_carteira.bfill()
cotacoes_carteira.info()

### Vamos ver como que as ações foram individualmente

In [ ]:
carteira_norm = cotacoes_carteira / cotacoes_carteira.iloc[0]
carteira_norm.plot(figsize=(15, 5))
plt.legend(loc='upper left')
plt.show()

### Vamos puxar o IBOV para comparar

In [ ]:
cotacao_ibov = yf.download('^BVSP', start, end)
display(cotacao_ibov)

### Criando um dataframe da Carteira com as quantidades de ações

In [ ]:
valor_investido = pd.DataFrame()

for ativo in carteira['Ativos']:
    valor_investido[ativo] = cotacoes_carteira[ativo] * carteira.loc[carteira['Ativos']==ativo, 'Qtde'].values[0]
    
display(valor_investido)

### Comparação Carteira x IBOV

In [ ]:
valor_investido['Total'] = valor_investido.sum(axis=1)

valor_investido_norm = valor_investido / valor_investido.iloc[0]
cotacao_ibov_norm = cotacao_ibov / cotacao_ibov.iloc[0]

valor_investido_norm['Total'].plot(figsize=(15, 5), label='Carteira')
cotacao_ibov_norm['Adj Close'].plot(label='IBOV')
plt.legend()
plt.show()

In [ ]:
retorno_carteira = valor_investido['Total'][-1] / valor_investido['Total'][0] - 1
retorno_ibov = cotacao_ibov['Adj Close'][-1] / cotacao_ibov['Adj Close'][0] - 1
print('Retorno carteira: {:.2%}'.format(retorno_carteira))
print('Retorno IBOV: {:.2%}'.format(retorno_ibov))

### Correlação da Carteira com o IBOV

In [ ]:
correlacao = valor_investido['Total'].corr(cotacao_ibov['Adj Close'])
print(correlacao)